# Redundant Internal Coordinates and B-matrix
#### This is a tutorial for the conversion to and from internal coordinates using the B matrix.

Redundent Internal Coordiantes, as proposed by Bakken and Helgaker, are defined using stretches, bends, and dihedral angles. Including vibrational motions, there are up to 3N-6 degrees of freedom for a molecule. Defining water with internal coordinates, there are two stretches, and one bend this defining all degrees of freedom. For molecules with more than three atoms, the number of internal coordiantes quickly becomes greater than the number of degrees of freedom and thus redundant.  

In order to convert cartesian coordiantes to internal coordinates, Wilson B matrix is used. The B matrix is defined as the derivative of the internal coordinates with respect to cartesian coordinates, and has dimensions of internal coordinates by 3 * N. 

$$\textbf {B}_{ij} = \frac{\delta q_i}{\delta x_j}$$

Simple rearrangment allows for the conversion between small changes in cartesian geometry to internal coordinates
$$\textbf {B}_{ij} \delta x = \delta q$$

This does not allow for the direct conversion between the geometry in cartesians and in internals. However, this isn't particularly helpful, since our goal is usually to conver a change in internal coordinates to cartesians which is covered later in discussing the back transformation. To create the internal geometry, we need to calculate the distances and angles using simple geometry.

Lets start by defining a simple cartesian geometry for ammonia and then defining all of the internal coordinates for the molecule. Note that the number of internal coordinates is greater than the number of degrees of freedom.

In [ ]:
import psi4
import numpy as np
from optking import printTools
printTools.printInit(printTools.cleanPrint)
printTools.print_opt("hi")

#a starting geometry can be taken from pubchem
mol = psi4.geometry("""
  pubchem:ammonium
""")
mol.set_multiplicity (1)
mol.set_molecular_charge(1)

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom)


from optking import stre, bend
r1 = stre.STRE(0,1)
r2 = stre.STRE(0,2)
r3 = stre.STRE(0,3)
r4 = stre.STRE(0,4)
b1 = bend.BEND(1,0,2)
b2 = bend.BEND(2,0,3)
b3 = bend.BEND(3,0,4)
b4 = bend.BEND(4,0,1)
b5 = bend.BEND(4,0,2)
b6 = bend.BEND(3,0,1)
intcos = (r1, r2, r3, r4, b1, b2, b3, b4, b5, b6)

print ("\nList of internal Coordinates in Bohr and Radians")
for i in range (len(intcos)):
    #q(xyzGeom) simply calls v3d.dist() for the two atoms in a stretch, and v3d.angle() for the three atoms in a bond angle
    print (" %d %s %.16f" % (i + 1, intcos[i], intcos[i].q(xyzGeom)))

#create an array for geometry in internals
qGeom = np.zeros(len(intcos), float)
for i in range (len(intcos)):
    qGeom[i] = intcos[i].q(xyzGeom)

print("\n Geometry in internals")    
printTools.printArray(qGeom)    

In [ ]:
from optking import intcosMisc
Bmatrix = intcosMisc.Bmat(intcos, xyz)
print('B matrix')
print(Bmatrix)

Our goal is to calculate the step in internal coordintes, this requires that either the forces or gradient are converted into internals.

In order to convert between the gradient or forces we need to introduce $\delta E$ luckily this easily simplifies to the form we need

$$\frac{\textbf {B}}{\delta q} = (\delta x)^{-1} \equiv \frac{\delta q}{\delta x} \frac{\delta E}{\delta q} = \frac{\delta E}{\delta x} \equiv \textbf{B}^T g_q = g_x$$ 

First lets get the gradient from Psi4, which is in cartesian

In [ ]:
psi4gradientMatrix = psi4.gradient('hf')
g = np.array( psi4gradientMatrix )
print (g)